In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt

In [12]:
# Read CSV file into DataFrame df
df = pd.read_csv('heart disease classification dataset.csv', index_col=0)

# Show dataframe
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,male,3,145.0,233.0,1,0,150.0,0,2.3,0,0,1,yes
1,37,male,2,130.0,250.0,0,1,187.0,0,3.5,0,0,2,yes
2,41,female,1,130.0,204.0,0,0,172.0,0,1.4,2,0,2,yes
3,56,male,1,120.0,236.0,0,1,178.0,0,0.8,2,0,2,yes
4,57,female,0,NaN,354.0,0,1,163.0,1,0.6,2,0,2,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,female,0,140.0,241.0,0,1,123.0,1,0.2,1,0,3,no
299,45,male,3,110.0,264.0,0,1,132.0,0,1.2,1,0,3,no
300,68,male,0,144.0,193.0,1,1,141.0,0,3.4,1,2,3,no
301,57,male,0,NaN,131.0,0,1,115.0,1,1.2,1,1,3,no


In [30]:
# check for any null values in cols, replace w/ mean of col
for col in df:
    if col != 'sex' and col != 'target':
        df[col].fillna(df[col].mean())
    if col == 'sex':
        counts = df[col].value_counts().index.tolist()
        df[col].fillna(counts[0])
        
        

In [16]:
#data exploration
for col in df:
    if col != 'sex' and col != 'target':
        print("col name:", col)
        print("min:", df[col].min())
        print("max:", df[col].max())
        print("mean:", df[col].mean())
        print("std:", df[col].std())

col name: age
min: 29
max: 77
mean: 54.366336633663366
std: 9.082100989837858
col name: cp
min: 0
max: 3
mean: 0.966996699669967
std: 1.0320524894832992
col name: trestbps
min: 94.0
max: 200.0
mean: 131.7123745819398
std: 17.629031902488265
col name: chol
min: 126.0
max: 564.0
mean: 246.31788079470198
std: 51.908284932787794
col name: fbs
min: 0
max: 1
mean: 0.1485148514851485
std: 0.35619787492797594
col name: restecg
min: 0
max: 2
mean: 0.528052805280528
std: 0.525859596359298
col name: thalach
min: 71.0
max: 202.0
mean: 149.86577181208054
std: 22.563686960858593
col name: exang
min: 0
max: 1
mean: 0.32673267326732675
std: 0.46979446452231716
col name: oldpeak
min: 0.0
max: 6.2
mean: 1.0396039603960396
std: 1.1610750220686343
col name: slope
min: 0
max: 2
mean: 1.3993399339933994
std: 0.6162261453459631
col name: ca
min: 0
max: 4
mean: 0.7293729372937293
std: 1.0226063649693276
col name: thal
min: 0
max: 3
mean: 2.3135313531353137
std: 0.6122765072781412


In [31]:
#X,y split
X = df.iloc[:, :-1]
y = df['target']
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,male,3,145.0,233.0,1,0,150.0,0,2.3,0,0,1
1,37,male,2,130.0,250.0,0,1,187.0,0,3.5,0,0,2
2,41,female,1,130.0,204.0,0,0,172.0,0,1.4,2,0,2
3,56,male,1,120.0,236.0,0,1,178.0,0,0.8,2,0,2
4,57,female,0,NaN,354.0,0,1,163.0,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,female,0,140.0,241.0,0,1,123.0,1,0.2,1,0,3
299,45,male,3,110.0,264.0,0,1,132.0,0,1.2,1,0,3
300,68,male,0,144.0,193.0,1,1,141.0,0,3.4,1,2,3
301,57,male,0,NaN,131.0,0,1,115.0,1,1.2,1,1,3


In [32]:
#stand X
for col in X:
    X_stand = (X-X.mean())/X.std()

C:\Users\willi\AppData\Local\Temp/ipykernel_16732/3801256960.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_stand = (X-X.mean())/X.std()
